In [1]:
import pandas as pd
import numpy as np
import tables
from pyhdf import SD
import h5py
import os
import datetime as dt
import pyhdf.VS
import pyhdf
from pyhdf.HDF import HDF
import scipy.signal
import math

#for mapping
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [2]:
tables.file._open_files.close_all()

In [3]:
#check path and ensure you are in path nearby data folder
os.getcwd()

'D:\\E Contents'

In [4]:
#enter your path followed by folder name and print once to make sure only hdf files are there in folder, then proceed
folderpath = r"D:\E Contents\ADHM 3 months" # make sure to put the 'r' in front
filepaths  = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]

### Extraction Function
Calipso files are each approximately 400-500Mb for one transit. So given a time window there are multiple files. Keep all the files in one particular folder after downloading from the CALIPSO Earthdata search.

Using this function, for each file we extract the metadata and set index as the date and time at start of granule.
Then we extract the parameters. Some of the parameters may be multidimensional i.e. for each timestamp they will have multiple readings, for eg. Backscatter_532 for each timestamp has 583 values corresponding to different altitudes. This altitude data is given in the metadata. For multidimensional parameters we convert them into a dataframe and store the whole file as dataframe within dataframe. We also specificy the latitude and longitude box for data and truncate it. This is the primary step in extraction as each CALIPSO granule has all lat and long for a given transit, and its necessary to discard redundant data to save space.

We then run a loop to extract all the files, and save them in a newly created HDF file to access data much more easily. We append main data to the hdf file itself at each loop. We use the corresnponding key from metadata to be the granule key in our multi-index dataframe. For saving the first set of data make sure to delete other file for previous names(do this while overwriting same data again too). If you are downloading new data and want to append the data then remove the delete file line and continue appending. Metadata files are appended as dataframes and together converted to HDF again to save space.

For example I downloaded files covering the latitudes and longitudes of Ali, Devasthal, Hanle and merak for 6 month (over 30 granules when satellite passes over), and we extract the granules and truncate it to those coordinates and save it as truncated dataset and metadata. In the instance that you download and want to process the granules separately, i.e. 20 first then 20 next to save computing power, then use the append mode (Type 'a'). Otherwise use the write mode (Type 'w'). This will automatically load existing metadata also to append, whereas writing for the first time will delete previous written files of same name. Please be careful. Kindly change the file names accordingly before writing a new file.

To extract other HDF files that are not CALIPSO (since we have explicitly used column names and modified data), it is better to use SD.SD from pyhdf, and HDF from pyhDF.HDF to access the columns and fields in data and metadata. The documentation is provided online.

In [ ]:
tables.file._open_files.close_all()

def extract_meta(file_name):
    #Access Metadata methodology
    #create a HDF interface
    HDF_interface = HDF(file_name)

    #initialize the VS API over the file and return a VS instance using the vstart() function of the HDF interface.

    vs_interface = HDF_interface.vstart()

    #Next, I use attach() to retrieve the "metadata" VD instance.

    meta = vs_interface.attach("metadata")
    #Retrieve info about all vdata fields.
    field_infos = meta.fieldinfo()
    #retreieve all data values
    all_data = meta.read(meta._nrecs)[0]

    meta.detach() #terminate access to vdata
    
    #put everthing in a dictionary
    metadata_dictionary = {}
    field_name_index = 0
    for field_info, data in zip(field_infos, all_data):
        metadata_dictionary[field_info[field_name_index]] = data
        
        
    metadata_df = pd.Series(metadata_dictionary).to_frame().T #insert the metadata dictionary as rows i.e. columns will be diff attributes
    metadata_df.set_index('Date_Time_at_Granule_Start', inplace=True)
    
    return metadata_df

def extract_func(file,trunc_list):
    ext_SD = SD.SD(file)
    para_list = list(ext_SD.datasets().keys())

    #extract the file and its keys to form a dictionary.
    data_dict = {}
    for keys in para_list:
        data_dict[keys] = ext_SD.select(keys).get()

    #if any data is mutidimensional convert it into a dataframe with the suffix _df
    data_df_dict = {}
    for key in data_dict.keys():
        if len(data_dict[key][0]) != 1:
            data_df_dict[key + "_df"]= [pd.DataFrame(obj,columns=['val']) for obj in data_dict[key]]


    data_dict_flat = {}
    for key in data_dict.keys():
        if len(data_dict[key][0]) == 1:
            data_dict_flat[key] = data_dict[key].flatten()

    data_dict_final = data_dict_flat.copy()
    data_dict_final.update(data_df_dict)

    df_final = pd.DataFrame({k:list(v) for k,v in data_dict_final.items()})
    
    metadata_df = extract_meta(file)

    ###Truncation
    
    
    data_long_trunc = df_final[df_final['Longitude']>trunc_list[1]]
    data_long_trunc = data_long_trunc[data_long_trunc['Longitude']<trunc_list[3]]
    
    data_lat_trunc = data_long_trunc[data_long_trunc['Latitude']>trunc_list[0]]
    data_lat_trunc = data_lat_trunc[data_lat_trunc['Latitude']<trunc_list[2]]
    
    sizelist = [len(df_final),len(data_long_trunc),len(data_lat_trunc)]
    
#     data_lat_trunc['Total_Attenuated_Backscatter_532']['Altitudes'] = metadata_df['Lidar_Data_Altitudes']
    
    
    data_lat_trunc.reset_index(inplace=True)
    data_lat_trunc.drop("index",axis=1,inplace=True)

    return data_lat_trunc,metadata_df,sizelist

# Apart from extraction we need to extract and append accordingly based on the structure. data files appended with index
# whereas the metadata is appended together


if __name__ == "__main__":
    a_w_variable = input("Enter 'a' for append, 'w' for write:")
    
    if a_w_variable == 'w':
        metadata_df = pd.DataFrame()
        filepath = os.path.join(os.getcwd(),"FinalData_truncated.hdf")
        if os.path.exists(filepath) == True:
            os.remove(filepath)
    if a_w_variable == 'a':
        filepath = os.path.join(os.getcwd(),"Final_metadata.hdf")
        if os.path.exists(filepath) == True:
            metadata_df = pd.read_hdf("Final_metadata.hdf",'metadata')
    else:
        a_w_variable = input("Enter 'a' for append, 'w' for write:")
        
    folderpath = input("Enter data folder path:")
    print(folderpath)

    file_list = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]
       
    coord_list = [float(item) for item in input("Enter coordinates (order: minlat,minlong,maxlat,maxlong):").split(',')]
    
#     if the datafile exists then you will be increasing size in append mode even though with the same key the data
#     is overwritten (key deleted and written), hdf5 does not reclaim space so the size of the file keeps increasing.

        
    sizelist = []
    
    for filename in file_list:
        print(filename, end='\r')      
    # call the extrtion function on one file granule and then append them one by one using metadata key
        data_trunc_temp, metadata_temp,ind_sizelist = extract_func(filename,coord_list)
    #     data_trunc_df = pd.concat([data_trunc_df,data_trunc_temp], axis = 0,join='outer')

        metadata_df = pd.concat([metadata_df,metadata_temp], axis = 0,join='outer')
        data_trunc_temp.to_hdf("FinalData_truncated.hdf",metadata_temp.index[0],mode='a')
        sizelist.append(ind_sizelist)
        print(ind_sizelist)

    metadata_df.to_hdf("Final_metadata.hdf",'metadata',mode='w')
    print(sizelist)


### Read Extracted Data
Whatever is stored in the truncated data and metadata file can be read now. While reading the data itself the code performs certain operations like appending the altitude columns from metadata file into the Total_attenuated_backscatter_532, and we convert the datetime from CALIPSO format (in UTC) which starts from 1993 Jan 1 to readable datetime. 

Then we shuffle the dataframe and bring forward the things needed mainly like latitude, longitude, TAB_532, surface elevation. We also use parameters such as 'Number_Bins_Shift' and 'Shifted_Altitude' to correspondingly shift our altitude column for each data point. Our hdf files will now be transformed into easily accessible dataframe.

The code goes through each item in metadata index, which is the index key for HDF5 file of the truncated data, and we use pandas to read the HDF5 one by one. We print this index as we loop through them. Later when shifting altidudes we iterate through rows and print the index from iterrows to monitor code flow.

In [2]:
def read_data(datafile,metadatafile):
    temp_df = pd.DataFrame()

    metadata_df = pd.read_hdf(metadatafile,'metadata')
    
    def append_alt(x):
        x['Altitudes'] = metadata_df['Lidar_Data_Altitudes'].values.tolist()[0]
    
    def Converdate(inputtime):
        conv = dt.datetime(1993,1,1,0,0,0,0) + dt.timedelta(0,inputtime)
        return conv.isoformat()

    ###when you append specify groupname and use groupby then simpsons/trap scipy integration. (that is for avergaing?)
    ###for intergrating one df guy and put it as new column and store. simple. 
    
    
    
    for item in metadata_df.index:
        
        test = pd.read_hdf(datafile,item) #reading we have to specify filename and the particular key if multiple data is appended
        test['SrNo'] = test.index
        test['granule'] = [item.split('T')[0]]*len(test)
        test.set_index(['granule','SrNo'],inplace=True)
        
        temp_df = pd.concat([temp_df,test],axis=0,join='outer')
        print(item)
    
    
    #Placing important columns at the start
    temp = temp_df.pop('Total_Attenuated_Backscatter_532_df')
    temp_df.insert(0,'TAB_532', temp)
    
    
    temp_df['TAB_532'].apply(lambda x : append_alt(x)) #appends altitude array to each TAB dataframe
    
    temp_df['date_time'] = temp_df['Profile_Time'].apply(lambda x : Converdate(x))
    
    temp = temp_df.pop('date_time')
    temp_df.insert(0,'date_time', temp)
    
    
    temp = temp_df.pop('Latitude')
    temp_df.insert(1,'Latitude', temp)
    
    
    temp = temp_df.pop('Longitude')
    temp_df.insert(2,'Longitude', temp)
    
    
    #shifting the appended altitude array with number bins shift and surface altitude shift based on their definition given in
    #the CALIPSO documentation, for each datapoint the altitudes within TAB_532 will be correspondingly shifted
    #based on these two columns given in Level 1 Data CALIPSO
    shifted_altitudes = []

    for row in temp_df[['TAB_532','Number_Bins_Shift','Surface_Altitude_Shift']].iterrows():
        print(row[0],end='\r')
        row[1]['TAB_532']['Altitudes'] = row[1]['TAB_532']['Altitudes'].shift(periods=row[1]['Number_Bins_Shift'])
        row[1]['TAB_532']['Altitudes'] = row[1]['TAB_532']['Altitudes'].apply(lambda x: x + row[1]['Surface_Altitude_Shift'])
#         print(row[1]['TAB_532'])
        temp_df.at[row[0],'TAB_532'] = row[1]['TAB_532']

    return temp_df,metadata_df


#The function is called to read truncated dataset and metadata in data_df and metadata_df
if __name__ == "__main__":
    data_df,metadata_df = read_data('FinalData_truncated.hdf','Final_metadata.hdf')

NameError: name 'pd' is not defined


#### This is a function to use bit encoded data in QC_Flag_2 parameter of CALIPSO to ensure we only select good data.
We form a new list called GB_list(Good/Bad), we convert the quality checks into 32bit string and check if any 1 exists 
in 11th to 16th bit (read left to right that is why we reverse), if yes we reject data (GB_list val is 1), then we check for 1's in consecutive timestamps i.e. 1250 timestamps. If a 1 in the bitstring (at any position) persists for many minutes then it is bad data. So we check how many integer value 0 are there in the next 1250 data and if 0 not in it then we reject data (GB_list value is 1). If none of this condition satisfied we accept data (GB_list val is 0). We are quality checking for approximately half a minute, less than the several minutes condition. One minute wil be 1250 other data.

In [1]:
def is_bad(qc_flag2):
#     print(range(len(qc_flag2)))
    GB_list = []
#     print(GB_list)
#     print(qc_flag2)
    qc_bits = list(map(lambda x:'{:032b}'.format(x),qc_flag2.values))
#     print(len(qc_bits))
#     print(qc_flag2.index.get_level_values(1))

    bandwith = 500
    for i in qc_flag2.index.get_level_values(1):
#         print(qc_flag2[i])
        bit_rev = '{:032b}'.format(qc_flag2[i])[::-1]
#         print(i,bit_rev)
        if '1' in bit_rev[10:15]:
            GB_list.append(1)
            continue
        if '1' in qc_bits[i]:
#             print(qc_bits[i])
            end_range = min(i+bandwith,len(qc_flag2)) #the next 500 timestamps or length of list whichever shorter is investigated
#             print(i,end_range)
            check_list_num = list(qc_flag2[i:end_range].values)
#             print(check_list_num)
            
            if 0 in check_list_num:
                GB_list.append(0)
            else:
                GB_list.append(1)
        else:
            GB_list.append(0)

    return GB_list
        
    

#### Averaging the backscatter data
Function that averages a singledate file given a dataframe and binsize in lat/longitude degrees for averaging
Horizontal averagin to remove SNR.
Given a binsize(in latitude degrees), we subtract and add that to the minimum and maximum values of latitude. Then we make a list of tuples containing latitude interval bins and make it the Interval Index of the dataframe.

In [ ]:


def group_hor_mean(df_singledate, binsize):
#     binrange = (30.5,32.5)
#     binsize = 0.25

    def custom_round(x, base):
        return base * round(float(x)/base)

# you can replace the base parameter with the nearest rounding you need  
#     print(len(df_singledate))

    
    
#     print(df_singledate['Latitude'].min(),df_singledate['Latitude'].max())

# Creating a bin list with given binsize that will be used to create an interval index
    a = custom_round(df_singledate['Latitude'].min(),binsize)
    b = custom_round(df_singledate['Latitude'].max(),binsize)
    
    if (a>df_singledate['Latitude'].min()):
        a = a - binsize
    
    if (b<df_singledate['Latitude'].max()):
        b = b + binsize
        
        
    bins = []
    temp = float('-inf')
    while (temp < b):
        temp = a + binsize
        bins.append((a,temp))
        a = temp
    
#     print(bins)
    list_of_bins = pd.IntervalIndex.from_tuples(bins) #make interval index from bins list
    
#     print(list_of_bins)
    
    
    df_singledate['Latitude_bins'] = pd.cut(df_singledate['Latitude'], list_of_bins) #segments or sorts data values into bins
    df_singledate = df_singledate.dropna(subset = ['Latitude_bins'])
  

    list_of_means = []
    nodata_list = []
#     s = 0
    #groupby and average data based on their latitude bins column
    for group_name, group_data in df_singledate.groupby(['Latitude_bins']):
        #some datapoints may be removed in quality check, hence those datapoints are not considered
        if(group_data['TAB_532'].shape[0] == 0): 
            nodata_list.append(group_name)
            continue    #skip this loop 
#         s = s + len(group_data)
# We take averages of everything when grouped with latitude bins and then multiply that 
        long_list = [long for long in group_data['Longitude']]
        avg_long = np.average(long_list)
        lat_list = [lat for lat in group_data['Latitude']]
        avg_lat = np.average(lat_list)
        
       
        DN_list= [DN for DN in group_data['Day_Night_Flag']]
        DN_flag = np.average(DN_list)
        
        surf_ele = [elev for elev in group_data['Surface_Elevation']]
        avg_ele = np.average(surf_ele)
        
        dt_length = len(group_data['date_time'])
        avg_datetime = dt.datetime.fromtimestamp(sum(map(dt.datetime.timestamp,map(dt.datetime.fromisoformat,group_data['date_time']))) / dt_length).isoformat()
#         print(avg_datetime)
        
        list_TAB_df = pd.concat(group_data['TAB_532'].values)
        mean_df = list_TAB_df.groupby(list_TAB_df.index).mean()
        lat_val = (group_name.left,group_name.right)
        
        mean_df['lat_bin_val'] = str(group_name)
        mean_df.set_index(['lat_bin_val',mean_df.index],inplace=True)
#         print(len(mean_df))
# Then we create a mean_dataframe that contains lat_bin as the index, other fields are repeated and we extract TAB_532 and append
# them as rows itself to later be useful when we use scipy peaks and have to do arithmeetic on it.
        mean_df['date_time'] = [avg_datetime] * len(mean_df)
        mean_df['Surface_elevation'] = [avg_ele]*len(mean_df)
        
        mean_df['Longitude'] = [avg_long] * len(mean_df)
        mean_df['Latitude'] = [avg_lat] * len(mean_df)
        mean_df['DN_Flag'] = [DN_flag]*len(mean_df)

    
        list_of_means.append(mean_df)

    mean_TAB_532 = pd.concat(list_of_means)
#     print(s)
    return mean_TAB_532, nodata_list
    

Function to act on all files instead of singledate and return them in form a dataframe that contains averaged and quality
checked data alone. The mean data will have 583 indexed rows (corresponding to TAB Altitudes), within each date (level 0 index) and latitude bin range(level 1 index). We go through each date and make bins and average accordingly and append 583 rows at each date and each bin

In [ ]:


def get_mean_df(data_df,binsize): 
    data_df['GB_2'] = is_bad(data_df['QC_Flag_2']) #prepares a GB_2 column from function is_bad defined above
    list_of_means = []
    no_data = []
    empty_list = []
    for item in data_df.index.get_level_values(0).unique():
        print(len(data_df.loc[item]))
        df_singledate = data_df.loc[item][data_df.loc[item]['QC_Flag']==0] #checks quality flag 1
        print(len(df_singledate))
        df_singledate = df_singledate[df_singledate['GB_2']==0] #checks and includes data that only satisfy this
        print(len(df_singledate))

#         df_singledate = data_df.loc[item]

#         print(len(df_singledate))

        if(len(df_singledate)==0): #this is to later make sure we dont have index correesponding to singledate with NULL objects
            empty_list.append(item)
            continue

#         print(item)
        mean_TAB_532,nodata = group_hor_mean(df_singledate,binsize) #mention the latitude averaging
        list_of_means.append(mean_TAB_532)
        no_data.append(nodata)
    
#     print(len([date for date in data_df.index.get_level_values(0).unique() if date not in empty_list]),len(list_of_means))
    mean_QC = pd.concat(list_of_means, keys = [date for date in data_df.index.get_level_values(0).unique() if date not in empty_list])
    #if there is a dataframe of a whole day that is removed by quality check we throw away the date to avoid appending null error
                        
    return mean_QC,no_data


In [ ]:
#Based on the printed values we can adjust the number to check consistent QC_Flag_2 in is_bad function. If too many datasets are 
#getting removed, increase the bandwith from 500. 

In [ ]:
#Run the function on your data_df
mean_TAB,no_data_list = get_mean_df(data_df,0.1)
mean_TAB_2,no_data_list = get_mean_df(data_df,0.5)

In [ ]:
#do multiple averaging and save them in separate HDF to be read later.

In [ ]:
mean_TAB.to_hdf("QC_backscatter_mean_point_one.hdf",'QC_mean',mode='w')

In [ ]:
mean_TAB_2.to_hdf("QC_backscatter_mean_half_deg.hdf",'mean',mode = 'w')

#### Load saved averaged data ffor extraction of PBLH using MSD


In [ ]:
mean_QC_half_deg = pd.read_hdf("Backscatter_mean_half_deg.hdf",'QC_mean')

In [ ]:
mean_QC_point_one = pd.read_hdf("Backscatter_mean_point_one.hdf",'QC_mean')

In [ ]:
mean_QC_half_deg

### Extracting PBLH

To extract PBLH we look at only the altitudes which are relvant (300m from the surface to 5kms from the surface), we also use the data given to us by surface elevation to truncate each single_files (one date and one lat_bin) within that altitude range.

We then take the standard deviation with four consecutives backscatter signals and altitude grouped. We identify the peaks corresponding to the backscatter signal and the standard deviation data. We then only consider important peaks defined with a cutoff height of the mean value of backscatter and standard deviation peaks. 
We then get the list of altitudes that correspond to these important peaks in backscatter as well as standard deviation (altitude values fot std is averaged for 4 points). These lists are also sorted, then we find for each altitude value of standard deivation maximum the corresponding nearest altitude value of the backscatter peak maximum. For every alt_val from standard deviation we find the difference of TAB value altitudes and store it in candidates list for PBLH

In [ ]:
def extract_PBLH(single_file):
    
    surf_ele = single_file['Surface_elevation'].iloc[0]
    cloud_bin_factor = 10
    
    trunc_after=(single_file[single_file['Altitudes']<0].index[0]-(math.ceil(single_file['Surface_elevation'][0]/0.03))-cloud_bin_factor)
    trunc_before = trunc_after - 160
#160*333m ~ 5kms from the surface elevation value is the upper limit for identifying PBLH
    single_file = single_file.truncate(before=trunc_before,after=trunc_after)
    
    std_list = pd.Series(single_file['val'].groupby(single_file.index // 4).std())
    #max_bs_list = np.array(single_file['val'].groupby(single_file.index // 4).max())
    alt_list = pd.Series(single_file['Altitudes']).groupby(single_file.index // 4).mean()
    #alt_compare_list = pd.Series({group_name:group_data for group_name,group_data in single_file['Altitudes'].groupby(single_file.index // 4)})
    
    peaks, _ = scipy.signal.find_peaks(single_file['val'])
    var_peaks, _ = scipy.signal.find_peaks(std_list)
    
    imp_peaks = scipy.signal.find_peaks(single_file['val'],height=np.mean(np.sort(single_file['val'].iloc[peaks])))
    imp_var_peaks=scipy.signal.find_peaks(std_list,height=np.mean(np.sort(std_list.iloc[var_peaks])))
    
    TAB_peak_list = single_file['Altitudes'].iloc[imp_peaks[0]].iloc[np.argsort(single_file['val'].iloc[imp_peaks[0]])]
    var_peak_list = alt_list.iloc[imp_var_peaks[0]].iloc[np.argsort(alt_list.iloc[imp_var_peaks[0]])]
    
    PB_val = None
    
    
    candidate_PB_list = []
    for alt_val in var_peak_list:
#         left_range = alt_val - 0.12
#         right_range = alt_val + 0.12
        
        candidate_PB_diff = []
        for val in TAB_peak_list:
            candidate_PB_diff.append((val,abs(alt_val-val)))

        candidate_PB_list.append((alt_val,sorted(candidate_PB_diff, key = lambda x: x[1])[0]))
#     print(candidate_PB_diff)
         
    
#     print(sorted(candidate_PB_list, key = lambda x: x[1][0]))
        
    
    PB_val = sorted(candidate_PB_list, key = lambda x: x[1][0])[0][1][0] #first guy in list, second value in tuple, first value in tuple
                cc
            
    return (PB_val - surf_ele)
                
        
                
                
                

In [ ]:
#function to extract PBLH of different datetimes. 
def extract_df_PB(mean_df):

    PBLH_dict = {'date_time':[],'Lat':[],'Long':[],'PBLH':[],'Surface_elevation':[],'DN_Flag':[]}
    for date in mean_df.index.get_level_values(0).unique():
        for lat_bin in mean_df.loc[date].index.get_level_values(0).unique():
            print(date,lat_bin)
            PBLH = extract_PBLH(mean_df.loc[date].loc[lat_bin])
            #print(PBLH)
            PBLH_dict['PBLH'].append(PBLH)
            PBLH_dict['date_time'].append(mean_df.loc[date].loc[lat_bin]['date_time'].iloc[0])
            PBLH_dict['Lat'].append(mean_df.loc[date].loc[lat_bin]['Latitude'].iloc[0])
            PBLH_dict['Long'].append(mean_df.loc[date].loc[lat_bin]['Longitude'].iloc[0])
            PBLH_dict['Surface_elevation'].append(mean_df.loc[date].loc[lat_bin]['Surface_elevation'].iloc[0])
            PBLH_dict['DN_Flag'].append(mean_df.loc[date].loc[lat_bin]['DN_Flag'].iloc[0])
            
    PBLH_df = pd.DataFrame(PBLH_dict)
    return PBLH_df
        

In [ ]:
PBLH_df_half_deg = extract_df_PB(mean_QC_half_deg)

In [ ]:
PBLH_df_point_one = extract_df_PB(mean_QC_point_one)

In [ ]:
PBLH_df_point_one['date_time']=PBLH_df_point_one['date_time'].apply(lambda x : dt.datetime.fromisoformat(x))

In [ ]:
PBLH_df_half_deg['date_time']=PBLH_df_half_deg['date_time'].apply(lambda x : dt.datetime.fromisoformat(x))

In [ ]:
#Writing PBLH data as CSV
def readable_data(PBLH_df,latlong,filename):
    zone_df = PBLH_df.query(str(latlong[0])+'< Lat <'+str(latlong[1])).query(str(latlong[2])+'< Long <'+str(latlong[3]))
    def dn_convert(dn_val):
        if dn_val == 0:
            return 'Day'
        else:
            return 'Night'
        
    zone_df['Day/Night'] = zone_df['DN_Flag'].map(dn_convert)
    zone_df.to_csv(filename, index = False,sep='\t')

In [ ]:
##Enter the latitude and longitudes to query within which to query (we will use boxes set 1degree by 1degree around)
hanle_coords = [32.28,33.28,78.46,79.46]
merak_coords = [33.30,34.30,78.12,79.12]
ali_coords = [32.13,33.13,79.50,80.50]
devasthal_coords = [28.86,29.86,79.18,80.18]


In [ ]:
#We take one degree by 1degree box around Hanle and print the values that come within this box
readable_data(PBLH_df_half_deg,hanle_coords,"Hanle_half_deg_Jan_Jul_2012.csv")
readable_data(PBLH_df_point_one,hanle_coords,"Hanle_point_one_Jan_Jul_2012.csv")

#We take one degree by 1degree box around merak and print the values that come within this box
readable_data(PBLH_df_half_deg,merak_coords,"merak_half_deg_Jan_Jul_2012.csv")
readable_data(PBLH_df_point_one,merak_coords,"merak_point_one_Jan_Jul_2012.csv")

#We take one degree by 1degree box around ali and print the values that come within this box
readable_data(PBLH_df_half_deg,ali_coords,"ali_half_deg_Jan_Jul_2012.csv")
readable_data(PBLH_df_point_one,ali_coords,"ali_point_one_Jan_Jul_2012.csv")

#We take one degree by 1degree box around devasthal and print the values that come within this box
readable_data(PBLH_df_half_deg,devasthal_coords,"devasthal_half_deg_Jan_Jul_2012.csv")
readable_data(PBLH_df_point_one,devasthal_coords,"devasthal_point_one_Jan_Jul_2012.csv")

In [ ]:
hanle_df_night_p1 = PBLH_df_point_one[PBLH_df_point_one['DN_Flag']==1].query(str(hanle_coords[0])+'< Lat <'+str(hanle_coords[1])).query(str(hanle_coords[2])+'< Long <'+str(hanle_coords[3]))
hanle_df_day_p1 = PBLH_df_point_one[PBLH_df_point_one['DN_Flag']==0].query(str(hanle_coords[0])+'< Lat <'+str(hanle_coords[1])).query(str(hanle_coords[2])+'< Long <'+str(hanle_coords[3]))

hanle_df_day_h = PBLH_df_half_deg[PBLH_df_half_deg['DN_Flag']==0].query(str(hanle_coords[0])+'< Lat <'+str(hanle_coords[1])).query(str(hanle_coords[2])+'< Long <'+str(hanle_coords[3]))
hanle_df_night_h = PBLH_df_half_deg[PBLH_df_half_deg['DN_Flag']==1].query(str(hanle_coords[0])+'< Lat <'+str(hanle_coords[1])).query(str(hanle_coords[2])+'< Long <'+str(hanle_coords[3]))

In [ ]:
merak_df_day_p1 = PBLH_df_point_one[PBLH_df_point_one['DN_Flag']==0].query(str(merak_coords[0])+'< Lat <'+str(merak_coords[1])).query(str(merak_coords[2])+'< Long <'+str(merak_coords[3]))
merak_df_night_p1 = PBLH_df_point_one[PBLH_df_point_one['DN_Flag']==1].query(str(merak_coords[0])+'< Lat <'+str(merak_coords[1])).query(str(merak_coords[2])+'< Long <'+str(merak_coords[3]))

merak_df_day_h = PBLH_df_half_deg[PBLH_df_half_deg['DN_Flag']==0].query(str(merak_coords[0])+'< Lat <'+str(merak_coords[1])).query(str(merak_coords[2])+'< Long <'+str(merak_coords[3]))
merak_df_night_h = PBLH_df_half_deg[PBLH_df_half_deg['DN_Flag']==1].query(str(merak_coords[0])+'< Lat <'+str(merak_coords[1])).query(str(merak_coords[2])+'< Long <'+str(merak_coords[3]))

In [ ]:
ali_df_day_p1 = PBLH_df_point_one[PBLH_df_point_one['DN_Flag']==0].query(str(ali_coords[0])+'< Lat <'+str(ali_coords[1])).query(str(ali_coords[2])+'< Long <'+str(ali_coords[3]))
ali_df_night_p1 = PBLH_df_point_one[PBLH_df_point_one['DN_Flag']==1].query(str(ali_coords[0])+'< Lat <'+str(ali_coords[1])).query(str(ali_coords[2])+'< Long <'+str(ali_coords[3]))

ali_df_day_h = PBLH_df_half_deg[PBLH_df_half_deg['DN_Flag']==0].query(str(ali_coords[0])+'< Lat <'+str(ali_coords[1])).query(str(ali_coords[2])+'< Long <'+str(ali_coords[3]))
ali_df_night_h = PBLH_df_half_deg[PBLH_df_half_deg['DN_Flag']==1].query(str(ali_coords[0])+'< Lat <'+str(ali_coords[1])).query(str(ali_coords[2])+'< Long <'+str(ali_coords[3]))

In [ ]:
devasthal_df_day_p1 = PBLH_df_point_one[PBLH_df_point_one['DN_Flag']==0].query(str(devasthal_coords[0])+'< Lat <'+str(devasthal_coords[1])).query(str(devasthal_coords[2])+'< Long <'+str(devasthal_coords[3]))
devasthal_df_night_p1 = PBLH_df_point_one[PBLH_df_point_one['DN_Flag']==1].query(str(devasthal_coords[0])+'< Lat <'+str(devasthal_coords[1])).query(str(devasthal_coords[2])+'< Long <'+str(devasthal_coords[3]))

devasthal_df_day_h = PBLH_df_half_deg[PBLH_df_half_deg['DN_Flag']==0].query(str(devasthal_coords[0])+'< Lat <'+str(devasthal_coords[1])).query(str(devasthal_coords[2])+'< Long <'+str(devasthal_coords[3]))
devasthal_df_night_h = PBLH_df_half_deg[PBLH_df_half_deg['DN_Flag']==1].query(str(devasthal_coords[0])+'< Lat <'+str(devasthal_coords[1])).query(str(devasthal_coords[2])+'< Long <'+str(devasthal_coords[3]))